In [ ]:
# ========================================================================================
# 🚀 RETAIL INVENTORY MANAGEMENT SYSTEM - AGILE IMPLEMENTATION USING LANGCHAIN
# ========================================================================================

# ========================================================================================
# 📦 SETUP AND IMPORTS
# ========================================================================================
!pip install -q langchain openai python-dotenv

import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

load_dotenv()

# Set API key manually (OPTION 2)
openai_api_key = ""
llm = ChatOpenAI(model="gpt-4o", temperature=0.3, openai_api_key=openai_api_key)

# ========================================================================================
# 🧠 AGENT CLASS DEFINITION
# ========================================================================================
class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(llm=llm, prompt=self.prompt, verbose=False)

    def respond(self, input_text, history):
        return self.chain.predict(input=input_text, conversation_history=history)

MAX_HISTORY_TOKENS = 8000

def manage_history(history):
    if len(history) > MAX_HISTORY_TOKENS:
        return history[-MAX_HISTORY_TOKENS:]
    return history

# ========================================================================================
# 🤖 AGENT ROLE PROMPTS
# ========================================================================================
role_prompts = {
    "Customer_Proxy": '''You are a Customer Proxy agent acting as a mediator between the retail stakeholder and the AI Scrum team.

Your role is to clearly relay inventory management expectations, goals, and user pain points to the Product Owner.

Your retail business needs an **AI-driven inventory management system** that:
1. Tracks inventory levels in real-time.
2. Predicts stock shortages and suggests reorder points.
3. Automates reordering processes based on demand forecasting.
4. Supports multiple warehouse locations.
5. Generates comprehensive reports on stock movement.

At the end of every sprint, you will review and provide feedback for potential changes or approvals.''',

    "Product_Owner": '''You are a Product Owner in an Agile Scrum team.

### Responsibilities:
- Translate the **Customer Proxy's** high-level business goals into user stories and backlog items.
- Prioritize and maintain the product backlog.
- Collaborate with all Scrum roles to define sprint goals.
- Communicate customer feedback to the team.
- Ensure all sprint deliverables meet customer expectations.

### Deliverables:
- Product backlog with user stories.
- Sprint plan (sprint goals, estimates, priority).
- Feedback response and change plan.''',

    "Scrum_Master": '''You are the Scrum Master, a facilitator of Agile ceremonies and responsible for removing blockers.

### Responsibilities:
- Facilitate daily standups, sprint planning, sprint reviews, and retrospectives.
- Ensure Scrum principles are followed.
- Help the team improve velocity and remove impediments.
- Promote collaboration and communication among all agents.

### Deliverables:
- Summary of Scrum ceremonies.
- Identified blockers and mitigation plans.''',

    "Business_Analyst": '''You are a Business Analyst collaborating with the Product Owner to refine user stories.

### Responsibilities:
- Break down product backlog items into detailed requirements.
- Include acceptance criteria, user personas, and edge cases.
- Clarify ambiguous user needs.

### Deliverables:
- Detailed functional and non-functional requirements.
- Workload estimates and productivity planning.''',

    "Architect": '''You are a System Architect designing scalable solutions.

### Responsibilities:
- Convert user stories into architecture components.
- Produce diagrams (components, APIs, DB schemas).
- Identify risks and technical constraints.

### Deliverables:
- Architectural blueprint.
- Design metrics and rework plan if needed.''',

    "Developer": '''You are a Developer responsible for delivering features as per the sprint backlog.

### Responsibilities:
- Write modular, testable code.
- Estimate lines of code (SLOC), productivity, and sprint duration.
- Collaborate with QA, UI/UX, and PO.

### Deliverables:
- Feature implementation.
- Sprint burndown progress report.''',

    "UI_UX_Designer": '''You are a UI/UX Designer focused on user-centric interface design.

### Responsibilities:
- Create wireframes and high-fidelity designs for prioritized user stories.
- Incorporate accessibility and responsiveness.
- Estimate design effort and timeline.

### Deliverables:
- Screens per sprint.
- Design feedback incorporated from reviews.''',

    "QA_Engineer": '''You are a QA Engineer responsible for validating sprint deliverables.

### Responsibilities:
- Develop test cases based on user stories.
- Execute functional, regression, and integration tests.
- Report bugs with severity levels.

### Deliverables:
- Test coverage metrics.
- QA sign-off or defect backlog.''',

    "DevOps_Engineer": '''You are a DevOps Engineer handling CI/CD pipelines and environment readiness.

### Responsibilities:
- Build CI/CD pipelines for testing and deployment.
- Define infrastructure as code (IaC).
- Monitor build and deployment success.

### Deliverables:
- Deployment plan per sprint.
- Productivity and environment uptime metrics.''',

    "Security_Engineer": '''You are a Security Engineer safeguarding the system from vulnerabilities.

### Responsibilities:
- Conduct sprint-based security audits.
- Perform threat modeling and code scanning.
- Recommend risk mitigation.

### Deliverables:
- Security report per sprint.
- Checklist of patches or concerns.''',

    "Technical_Writer": '''You are a Technical Writer responsible for documenting sprint deliverables.

### Responsibilities:
- Write user guides, API docs, release notes.
- Collaborate with developers and testers for accuracy.
- Ensure documentation is clear and version-controlled.

### Deliverables:
- Updated product documentation.
- Summary of sprint releases.'''
}

agents = {role: Agent(role, prompt, llm) for role, prompt in role_prompts.items()}

/var/folders/ch/737dr97s4gs48czxfmlp_wr00000gn/T/ipykernel_81886/897558249.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0.3, openai_api_key=openai_api_key)
/var/folders/ch/737dr97s4gs48czxfmlp_wr00000gn/T/ipykernel_81886/897558249.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=llm, prompt=self.prompt, verbose=False)


In [2]:
# ========================================================================================
# 🧑‍💼 SPRINT EXECUTION (WITH MANUAL CUSTOMER INTERACTION)
# ========================================================================================
conversation_history = ""
print("\n================================================================================")
print("🧑‍💼 CUSTOMER REQUIREMENTS PHASE")
print("================================================================================")
initial_input = input("📥 Please enter the type of inventory system you want to build: ")
conversation_history = f"Customer_Proxy: {initial_input}\n"

sprint_order = [
    "Product_Owner", "Business_Analyst", "Architect", "Developer", "UI_UX_Designer",
    "QA_Engineer", "DevOps_Engineer", "Security_Engineer", "Technical_Writer",
    "Scrum_Master", "Customer_Proxy"
]

for role in sprint_order:
    print("\n" + "=" * 80)
    print(f"🧑‍💻 {role.upper()} PHASE")
    print("=" * 80)
    task_prompt = role_prompts[role]
    try:
        if role == "Customer_Proxy":
            print("\n📥 Awaiting feedback from the customer...")
            response = input("🗣️  Enter feedback from Customer Proxy (e.g., 'Looks good', 'Please rework feature X'): ")

        elif role == "Technical_Writer":
            print("\n📘 Estimating documentation tasks based on standard productivity rates...")
            user_doc_pages = 147
            defects_found = 163
            response = f"""
### Documentation Plan

- **User Documentation**: {user_doc_pages} pages  
- **Writing Productivity**: 4 pages/hour  
- **Estimated Writing Time**: {user_doc_pages / 4:.2f} hours

---

### Review Process

- **Review Preparation**: {user_doc_pages} pages @ 4 pages/hour = {user_doc_pages / 4:.2f} hours  
- **Review Meeting**: {user_doc_pages} pages @ 6 pages/hour = {user_doc_pages / 6:.2f} hours  
- **Defects Found**: {defects_found}  
- **Rework Productivity**: 8 defects/hour  
- **Rework Time**: {defects_found / 8:.2f} hours

---

### Total Documentation Effort: ~{(user_doc_pages / 4 * 2 + user_doc_pages / 6 + defects_found / 8):.2f} hours"""

        else:
            response = agents[role].respond(task_prompt, conversation_history)

        print("-" * 80)
        print(f"🤖 {role} RESPONSE:\n{response}")
        print("=" * 80)

        conversation_history += f"{role}: {response}\n"
        conversation_history = manage_history(conversation_history)

        if role == "Customer_Proxy":
            if any(x in response.lower() for x in ["approved", "looks good", "happy"]):
                print("✅ Customer approved. Sprint is complete.")
                break
            elif any(x in response.lower() for x in ["change", "adjust", "rework"]):
                print("🔁 Customer requested rework. Restarting QA phase.")
                sprint_order.insert(sprint_order.index("QA_Engineer"), "QA_Engineer")

    except Exception as e:
        print(f"❌ Error during {role} execution: {e}")
        break

# ========================================================================================
# 📘 END OF SPRINT
# ========================================================================================
print("\n✅ SPRINT WORKFLOW COMPLETED.")



🧑‍💼 CUSTOMER REQUIREMENTS PHASE

🧑‍💻 PRODUCT_OWNER PHASE
--------------------------------------------------------------------------------
🤖 Product_Owner RESPONSE:
### Product Backlog with User Stories

1. **User Story 1: Ergonomic Design**
   - As a customer, I want the chair to have an ergonomic design so that I can sit comfortably for long hours without experiencing back pain.
   - Acceptance Criteria:
     - Adjustable lumbar support
     - Contoured seat cushion
     - Adjustable armrests

2. **User Story 2: Material Quality**
   - As a customer, I want the chair to be made of high-quality materials so that it is durable and long-lasting.
   - Acceptance Criteria:
     - Use of breathable mesh fabric
     - Sturdy metal frame
     - High-density foam padding

3. **User Story 3: Aesthetic Appeal**
   - As a customer, I want the chair to have a modern and stylish appearance so that it complements my office decor.
   - Acceptance Criteria:
     - Sleek design with a variety of color